<a href="https://colab.research.google.com/github/stxupengyu/MF-for-Movie-Recommendation/blob/main/movie_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#link google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#enter the folder
!ls
import os
os.chdir('/content/drive/My Drive/Project/recsys')
!ls

drive  sample_data
'movie_data(大于等于20).csv'   rating.csv       微信图片_20201105195005.gdoc
 movie_recomendation.ipynb     rating_new.csv   微信图片_20201105195005.jpg


In [ ]:
#构造数据集
!pip install scikit-surprise
import numpy as np
import pandas as pd
from surprise import SVD,SVDpp,NormalPredictor,NMF,KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold, split
#读取数据
origin_data=pd.read_csv('movie_data(大于等于20).csv',encoding="unicode_escape") 
origin_data=origin_data.iloc[:,[3,1,5]]
origin_data=origin_data.dropna()
#store the dataset and then read it 
origin_data.to_csv('rating.csv',header=0,index=0)
# 指定文件所在路径
file_path = os.path.expanduser('rating.csv')
# 告诉文本阅读器，文本的格式是怎么样的
reader = Reader(line_format='user item rating', sep=',')
# 加载数据
data = Dataset.load_from_file(file_path, reader=reader)

     |████████████████████████████████| 11.8MB 2.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670899 sha256=96a7005ea12c2c31a9506dbc467c52d7e2c3296f094a3b2ee11cbe1dd846fa41
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
#使用基于用户相似度的协同过滤算法
sim_options = {'name': 'pearson_baseline'}
algo1 = KNNBasic(k=40, min_k=1,sim_options=sim_options)
#k近邻数量 sim_options利用的相似度标准

# 使用biasSVD
algo2 = SVD(biased=True,n_factors=100,n_epochs=20,lr_all=0.005,reg_all=0.02)
#biased是否使用biasedSVD，n_factors特征维度，n_epochs迭代次数,lr_all学习率,reg_all=0.02正则化系数

# Run 5-fold cross-validation and print results
algo1_result=cross_validate(algo1, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
algo2_result=cross_validate(algo2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8271  0.8335  0.8277  0.8255  0.8330  0.8294  0.0033  
MAE (testset)     0.6534  0.6574  0.6534  0.6514  0.6584  0.6548  0.0027  
Fit time          10.00   9.86    9.78    9.71    9.92    9.85    0.10    
Test time         13.51   13.80   13.69   13.81   13.59 

In [ ]:
#构造数据集
origin_data=pd.read_csv('rating.csv')
user = origin_data.iloc[:,0]
item = origin_data.iloc[:,1]
rating = origin_data.iloc[:,2]
user_num, item_num, none = origin_data.max()
#统计每名用户的评分总和
user2rating=np.zeros(int(user_num))
#统计每名用户的评价次数
user2times=np.zeros(int(user_num))
#统计每名用户评价过的物品
user2items = [[] for _ in range(int(user_num))]
for i in range(len(user)):
  user2rating[int(user[i])-1] = user2rating[int(user[i])-1] + rating[i]#这里的-1是因为用户index从1开始
  user2times[int(user[i])-1] = user2times[int(user[i])-1] + 1
  user2items[int(user[i])-1] = user2items[int(user[i])-1]+[(int(item[i])-1)]
#每名用户的平均评分
user2average=user2rating/user2times
k=0.01#补全矩阵系数
#生成每名用户将要评价的物品id
user2items_new = [[] for _ in range(int(user_num))]
#总的物品列表
item_list = [id for id in range(int(item_num))]
for id, inst in enumerate(user2items):
  temp_list=list(set(item_list).difference(set(inst)))  # item_list中有而inst中没有的
  temp_list=temp_list[:int(k*len(temp_list))]# 只选择其中的0.1份
  user2items_new[id] = user2items_new[id] + temp_list
#生成要补全的数组
generate_list=[]
for user_i in range(int(user_num)):
  for item_i in user2items_new[user_i]:
    generate_list.append([user_i+1, item_i+1, user2average[user_i]])#这里要+1 恢复由1开始的index
#将补全的数组和原始数据合并，并保存为文件
pd.DataFrame(np.vstack([origin_data.values,np.array(generate_list)])).to_csv('rating_new.csv',header=0,index=0)
# 指定文件所在路径
file_path = os.path.expanduser('rating_new.csv')
# 告诉文本阅读器，文本的格式是怎么样的
reader = Reader(line_format='user item rating', sep=',')
# 加载数据
data = Dataset.load_from_file(file_path, reader=reader)

In [ ]:
# 使用biasSVD
algo3 = SVD(biased=True,n_factors=100,n_epochs=20,lr_all=0.005,reg_all=0.02)
#biased是否使用biasedSVD，n_factors特征维度，n_epochs迭代次数,lr_all学习率,reg_all=0.02正则化系数

# Run 5-fold cross-validation and print results
algo3_result=cross_validate(algo3, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7444  0.7418  0.7452  0.7442  0.7435  0.7438  0.0012  
MAE (testset)     0.5657  0.5627  0.5662  0.5632  0.5642  0.5644  0.0014  
Fit time          13.81   13.72   13.83   13.77   13.83   13.79   0.04    
Test time         0.61    0.61    0.60    0.60    0.59    0.60    0.01    
